In [1]:
#OTP Verification System
#Import required Python libraries

import tkinter as tk #To create GUI(Graphical User Interface)
from tkinter import messagebox #To craete message box
import random #To create 6 digit OTP
import smtplib #To connect server to send OTP on email
from email.mime.text import MIMEText #To create text message GUI-with MIME-Multipurpose Internet Mail Extension
import threading #Threading allows you to have different parts of your process run concurrently.will not break program flow
import time #To genetate timer

# Define OTPVerification class and creating GUI
class OTPVerification:
    def __init__(self, attr):
        self.attr = attr
        self.attr.title("OTP Verification")#title of message box
        self.attr.geometry("400x200") 

        self.email_label = tk.Label(attr, text="Enter Email:")#Enter email in GUI
        self.email_label.pack()
        self.email_entry = tk.Entry(attr)
        self.email_entry.pack()

        self.send_otp_button = tk.Button(attr, text="Send OTP", command=self.send_otp) #define send OTP button
        self.send_otp_button.pack()

        self.otp_label = tk.Label(attr, text="Enter OTP:")#Enter OTP in GUI
        self.otp_label.pack()
        self.otp_entry = tk.Entry(attr)
        self.otp_entry.pack()

        self.verify_button = tk.Button(attr, text="Verify OTP", command=self.verify_otp)# define Verify OTP button
        self.verify_button.pack()
  
        self.generated_otp = None
        self.attempts = 0

        self.timer_label = tk.Label(attr, text="")
        self.timer_label.pack()

        self.timer_thread = None
        self.timer_running = False
        self.timer_duration = 120  # 2 minutes timer 
    
    # Timer countdown 
    def start_timer(self):
        self.timer_running = True
        remaining_time = self.timer_duration

        while remaining_time >= 0 and self.timer_running:
            mins, secs = divmod(remaining_time, 60)
            timer_str = '{:02d}:{:02d}'.format(mins, secs)
            self.timer_label.config(text="Time remaining: " + timer_str)
            self.timer_label.update()
            remaining_time -= 1
            time.sleep(1)

        if self.timer_running:
            self.timer_label.config(text="Time's up!")
            self.timer_label.update()
            messagebox.showinfo("Timeout", "OTP verification timeout!")
            self.reset_ui()
    #Send OTP to recipient email from sender email address
    def send_otp(self):
        email = self.email_entry.get()
        port=587
        passward=input("Enter passward")

        if email:
            self.generated_otp = str(random.randint(100000, 999999)) #Generating 6 digit OTP
            msg = MIMEText("Your OTP is: " + self.generated_otp)
            msg['Subject'] = 'OTP Verification'
            msg['From'] = 'palakpastagiya@gmail.com' #Sender email
            msg['To'] = email

            try:
                server = smtplib.SMTP('smtp.gmail.com', 587)  # SMTP server address and port
                server.starttls()#start the server
                server.login('palakpastagiya@gmail.com',passward )  
                server.sendmail('palakpastagiya@gmail.com', email, msg.as_string())# OTP sent to Sender to recepient email
                server.quit()
                messagebox.showinfo("Success", "OTP sent successfully!")
                if self.timer_thread: #As soon as OTP is sent timer will start
                    self.timer_running = False
                    self.timer_thread.join()
                self.timer_thread = threading.Thread(target=self.start_timer)
                self.timer_thread.start()
            except Exception as e:
                messagebox.showerror("Error", "Failed to send OTP. Error: {}".format(str(e)))
        else:
            messagebox.showerror("Error", "Please enter your email address!")
            
#Verifying OTP by matching user entered OTP and generated OTP
    def verify_otp(self):
        entered_otp = self.otp_entry.get()
        if entered_otp == self.generated_otp:
            messagebox.showinfo("Success", "OTP verified successfully!")
            self.reset_ui()
        else:
            self.attempts += 1 # count the attempts
            if self.attempts >= 3:#After 3 wrong OTP the session will be expired and we need to send OTP again
                messagebox.showerror("Error", "Maximum attempts reached. Session expired!")
                self.reset_ui()
            else:
                messagebox.showerror("Error", "Invalid OTP! {} attempts remaining.".format(3 - self.attempts))

       
#If OTP attempts 3 times are wrong it will restart timer and delete OTP 
    def reset_ui(self):
        self.otp_entry.delete(0, tk.END)
        if self.timer_thread:
            self.timer_running = False
            self.timer_thread.join()
        self.timer_label.config(text="")
        self.timer_label.update()
        self.attempts = 0
        
#The application is ready to run
def main():
    root = tk.Tk() #To create a main window, tkinter offers a method ‘Tk'
    app = OTPVerification(root)
    root.mainloop() # an infinite loop used to run the application, wait for an event to occur, and process the event as long as the window is not closed.

if __name__ == "__main__":
    main()